#### 1 - Localiza cada moeda, extraindo da imagem um array com cada uma das localizações
#### 2 - em cada recorte da imagem original, transforma em B&W, escala o recorte e extrai as features
#### 3 - passa cada um dos recortes para as MLP's e armazena o resultado de cada imagem
#### 4 - Avalia as probabilidades de cada resultado e informa o resultado com maior probabilidade como resposta

In [ ]:
###

In [7]:
import os
import cv2
import numpy as np
from minisom import MiniSom

path_images = 'coin_images/train/1-c/'
coin_img = '1-c_2002_10.jpg'

In [13]:
# Inicialize as listas para armazenar as características e os rótulos
features = []
labels = []

# Para cada diretório na pasta atual
for dir_name in [path_images]:#os.listdir('.'):
    if os.path.isdir(dir_name):

        # Para cada arquivo no diretório
        for file_name in os.listdir(dir_name):

            # Se o arquivo for uma imagem
            if (file_name.endswith('.png') or file_name.endswith('.jpg')) and file_name.endswith('_gs.jpg'):

                # Carregue a imagem em escala de cinza
                img_path = os.path.join(dir_name, file_name)
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Redimensione a imagem para 64x64 pixels
                image_resized = cv2.resize(image, (128, 128))

                # Achate a imagem em um vetor
                feature = image_resized.flatten()

                # Normalize as características para terem uma média de 0 e um desvio padrão de 1
                feature = (feature - np.mean(feature)) / np.std(feature)

                # Adicione as características e o rótulo às suas respectivas listas
                features.append(feature)
                labels.append(dir_name)  # assumindo que o nome do diretório é o rótulo

# Converta as listas em arrays numpy
features = np.array(features)
labels = np.array(labels)


In [14]:
# Inicialize um dicionário para armazenar os SOMs e seus rótulos
soms = {}
som_labels = {}
neurons = 50

# Obtenha os valores únicos de moedas
coin_values = np.unique(labels)

# Para cada valor de moeda
for value in coin_values:

    # Crie um novo SOM
    som = MiniSom(x=neurons, y=neurons, input_len=features.shape[1])

    # Treine o SOM com as características das moedas desse valor
    som.train_random(features[labels == value], num_iteration=5000)

    # Armazene o SOM
    soms[value] = som

    # Inicialize uma matriz para armazenar os rótulos do SOM
    som_labels[value] = np.empty((neurons, neurons), dtype=str)  # Mude para dtype=str

    # Para cada neurônio no SOM
    for i in range(neurons):
        for j in range(neurons):

            # Encontre as características que são mais semelhantes a esse neurônio
            min_dist = np.inf
            min_label = None
            for k, feature in enumerate(features[labels == value]):
                dist = np.linalg.norm(som.get_weights()[i, j] - feature)
                if dist < min_dist:
                    min_dist = dist
                    min_label = labels[labels == value][k]

            # Rotule o neurônio com a classe de moeda mais comum entre as características mais semelhantes
            som_labels[value][i, j] = min_label



AttributeError: 'MiniSom' object has no attribute 'weights'

In [17]:
# Inicialize uma matriz para armazenar os rótulos do SOM
som_labels[value] = np.empty((neurons, neurons), dtype=str)  # Mude para dtype=str

# Para cada neurônio no SOM
for i in range(neurons):
    for j in range(neurons):

        # Encontre as características que são mais semelhantes a esse neurônio
        min_dist = np.inf
        min_label = None
        for k, feature in enumerate(features[labels == value]):
            dist = np.linalg.norm(som.get_weights()[i, j] - feature)
            if dist < min_dist:
                min_dist = dist
                min_label = labels[labels == value][k]

        # Rotule o neurônio com a classe de moeda mais comum entre as características mais semelhantes
        som_labels[value][i, j] = min_label


In [18]:
def classify_coin(image, soms, som_labels):
    # Redimensione a imagem para 64x64 pixels
    image_resized = cv2.resize(image, (128, 128))

    # Achate a imagem em um vetor
    feature = image_resized.flatten()

    # Normalize as características para terem uma média de 0 e um desvio padrão de 1
    feature = (feature - np.mean(feature)) / np.std(feature)

    # Inicialize uma variável para armazenar a menor distância e a classe correspondente
    min_dist = np.inf
    min_class = None

    # Para cada SOM
    for value, som in soms.items():

        # Encontre o neurônio mais semelhante
        winner = som.winner(feature)

        # Calcule a distância entre a característica e o neurônio vencedor
        dist = np.linalg.norm(som.get_weights()[winner] - feature)


        # Se a distância for menor que a menor distância encontrada até agora
        if dist < min_dist:

            # Atualize a menor distância e a classe correspondente
            min_dist = dist
            min_class = som_labels[value][winner]

    # Retorne a classe com a menor distância
    return min_class


In [19]:
classify_coin(cv2.imread(path_images + coin_img, cv2.IMREAD_GRAYSCALE), soms, som_labels)

'c'

In [20]:
classify_coin(cv2.imread('coin_images/train/1-c/1-c_1994_4_rot90.jpg', cv2.IMREAD_GRAYSCALE), soms, som_labels)

'c'

In [21]:
classify_coin(cv2.imread('coin_images/train/5-c/5-c_2009_17.jpg', cv2.IMREAD_GRAYSCALE), soms, som_labels)

'c'

In [23]:
som_labels

{'coin_images/train/1-c/': array([['c', 'c', 'c', ..., 'c', 'c', 'c'],
        ['c', 'c', 'c', ..., 'c', 'c', 'c'],
        ['c', 'c', 'c', ..., 'c', 'c', 'c'],
        ...,
        ['c', 'c', 'c', ..., 'c', 'c', 'c'],
        ['c', 'c', 'c', ..., 'c', 'c', 'c'],
        ['c', 'c', 'c', ..., 'c', 'c', 'c']], dtype='<U1')}